### Imports and Utils

In [3]:
!pip install -q git+https://www.github.com/google/neural-tangents

In [4]:
import jax.numpy as np

from jax import random
from jax.experimental import optimizers
from jax.api import jit, grad, vmap

import functools



import neural_tangents as nt
from neural_tangents import stax

In [5]:
import tensorflow as tf

from sklearn import datasets
from sklearn.model_selection import train_test_split

import numpy as np

diabetes = datasets.load_boston()
X, y = diabetes.data, diabetes.target

y = np.reshape(y, (-1,1))

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.1, random_state=13)

#y_train = y_train /max(y_train)
#y_test = y_test/max(y_test)

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(455, 13)
(51, 13)
(455, 1)
(51, 1)


In [7]:
from neural_tangents import stax

n = 2048

def WideResNetBlock(n):
   Main = stax.serial(stax.Relu(), stax.Dense(n, W_std=1.5, b_std=0.05))
   Shortcut = stax.Identity()
   return stax.serial(stax.FanOut(2),
                     stax.parallel(Main, Shortcut),
                     stax.FanInSum())

init_fn, apply_fn, kernel_fn = stax.serial(stax.Dense(n, W_std=1.5, b_std=0.05), 
                                           WideResNetBlock(n),
                                           WideResNetBlock(n),                                                                                                                          
                                           stax.Relu(),
                                           stax.Dense(1, W_std=1.5, b_std=0.05))

apply_fn = jit(apply_fn)
kernel_fn = jit(kernel_fn, static_argnums=(2,))

In [9]:
predict_fn = nt.predict.gradient_descent_mse_ensemble(kernel_fn, X_train, 
                                                      y_train, diag_reg=1e-1)

ntk_prediction, _ = predict_fn(x_test=X_test, get='ntk', compute_cov=True)


from sklearn.metrics import mean_squared_error

print(mean_squared_error(ntk_prediction, y_test)/y_test.shape[0])

0.7708018299466072


In [10]:
from neural_tangents import stax

n = 2048


Main1 = stax.serial(stax.Relu(), stax.Dense(n, W_std=1.5, b_std=0.05))

Main2 = stax.serial(Main1, stax.Relu(), stax.Dense(n, W_std=1.5, b_std=0.05))


S3Net =  stax.serial(stax.FanOut(4), stax.parallel(Main1, Main2), stax.FanInSum())

init_fn, apply_fn, kernel_fn =  stax.serial(stax.Dense(n, W_std=1.5, b_std=0.05), S3Net,
                                            stax.Relu(),
                                            stax.Dense(1, W_std=1.5, b_std=0.05))

apply_fn = jit(apply_fn)
kernel_fn = jit(kernel_fn, static_argnums=(2,))

In [11]:
predict_fn = nt.predict.gradient_descent_mse_ensemble(kernel_fn, X_train, 
                                                      y_train, diag_reg=1e-1)

ntk_prediction, _ = predict_fn(x_test=X_test, get='ntk', compute_cov=True)


from sklearn.metrics import mean_squared_error

print(mean_squared_error(ntk_prediction, y_test)/y_test.shape[0])

/usr/local/lib/python3.6/dist-packages/jax/_src/lax/lax.py:6081: UserWarning: Explicitly requested dtype <class 'jax._src.numpy.lax_numpy.float64'> requested in astype is not available, and will be truncated to dtype float32. To enable more dtypes, set the jax_enable_x64 configuration option or the JAX_ENABLE_X64 shell environment variable. See https://github.com/google/jax#current-gotchas for more.
  warnings.warn(msg.format(dtype, fun_name , truncated_dtype))


0.7514009825834236


/usr/local/lib/python3.6/dist-packages/neural_tangents/stax.py:3926: UserWarning: `FanIn` layers assume independent inputs which is not verified in the code. Please make sure to have at least one `Dense` / `Conv` / `GlobalSelfAttention` etc. layer in each branch.
  warnings.warn('`FanIn` layers assume independent inputs which is not verified'


In [12]:
from neural_tangents import stax

n = 2048

def WideResNetBlock(n):
   Main = stax.serial(stax.Relu(), stax.Dense(n, W_std=1.5, b_std=0.05))
   Shortcut = stax.Identity()
   return stax.serial(stax.FanOut(2),
                     stax.parallel(Main, Shortcut),
                     stax.FanInSum())

init_fn, apply_fn, kernel_fn = stax.serial(stax.Dense(n, W_std=1.5, b_std=0.05), 
                                           WideResNetBlock(n),
                                           WideResNetBlock(n),
                                           WideResNetBlock(n),                                                                                                                      
                                           stax.Relu(),
                                           stax.Dense(1, W_std=1.5, b_std=0.05))

apply_fn = jit(apply_fn)
kernel_fn = jit(kernel_fn, static_argnums=(2,))

In [13]:
predict_fn = nt.predict.gradient_descent_mse_ensemble(kernel_fn, X_train, 
                                                      y_train, diag_reg=1e-1)

ntk_prediction, _ = predict_fn(x_test=X_test, get='ntk', compute_cov=True)


from sklearn.metrics import mean_squared_error

print(mean_squared_error(ntk_prediction, y_test)/y_test.shape[0])

/usr/local/lib/python3.6/dist-packages/jax/_src/lax/lax.py:6081: UserWarning: Explicitly requested dtype <class 'jax._src.numpy.lax_numpy.float64'> requested in astype is not available, and will be truncated to dtype float32. To enable more dtypes, set the jax_enable_x64 configuration option or the JAX_ENABLE_X64 shell environment variable. See https://github.com/google/jax#current-gotchas for more.
  warnings.warn(msg.format(dtype, fun_name , truncated_dtype))
/usr/local/lib/python3.6/dist-packages/neural_tangents/stax.py:3926: UserWarning: `FanIn` layers assume independent inputs which is not verified in the code. Please make sure to have at least one `Dense` / `Conv` / `GlobalSelfAttention` etc. layer in each branch.
  warnings.warn('`FanIn` layers assume independent inputs which is not verified'


0.7462948944894423


In [14]:
from neural_tangents import stax

n = 2048


Main1 = stax.serial(stax.Relu(), stax.Dense(n, W_std=1.5, b_std=0.05))

Main2 = stax.serial(Main1, stax.Relu(), stax.Dense(n, W_std=1.5, b_std=0.05))

Main3 = stax.serial(Main2, stax.Relu(), stax.Dense(n, W_std=1.5, b_std=0.05))

S3Net =  stax.serial(stax.FanOut(4), stax.parallel(Main1, Main2, Main3), stax.FanInSum())

init_fn, apply_fn, kernel_fn =  stax.serial(stax.Dense(n, W_std=1.5, b_std=0.05), S3Net,
                                            stax.Relu(),
                                            stax.Dense(1, W_std=1.5, b_std=0.05))

apply_fn = jit(apply_fn)
kernel_fn = jit(kernel_fn, static_argnums=(2,))

In [15]:
predict_fn = nt.predict.gradient_descent_mse_ensemble(kernel_fn, X_train, 
                                                      y_train, diag_reg=1e-1)

ntk_prediction, _ = predict_fn(x_test=X_test, get='ntk', compute_cov=True)


from sklearn.metrics import mean_squared_error

print(mean_squared_error(ntk_prediction, y_test)/y_test.shape[0])

/usr/local/lib/python3.6/dist-packages/jax/_src/lax/lax.py:6081: UserWarning: Explicitly requested dtype <class 'jax._src.numpy.lax_numpy.float64'> requested in astype is not available, and will be truncated to dtype float32. To enable more dtypes, set the jax_enable_x64 configuration option or the JAX_ENABLE_X64 shell environment variable. See https://github.com/google/jax#current-gotchas for more.
  warnings.warn(msg.format(dtype, fun_name , truncated_dtype))
/usr/local/lib/python3.6/dist-packages/neural_tangents/stax.py:3926: UserWarning: `FanIn` layers assume independent inputs which is not verified in the code. Please make sure to have at least one `Dense` / `Conv` / `GlobalSelfAttention` etc. layer in each branch.
  warnings.warn('`FanIn` layers assume independent inputs which is not verified'


0.7260892672407506


In [16]:
from neural_tangents import stax

n = 2048

def WideResNetBlock(n):
   Main = stax.serial(stax.Relu(), stax.Dense(n, W_std=1.5, b_std=0.05))
   Shortcut = stax.Identity()
   return stax.serial(stax.FanOut(2),
                     stax.parallel(Main, Shortcut),
                     stax.FanInSum())

init_fn, apply_fn, kernel_fn = stax.serial(stax.Dense(n, W_std=1.5, b_std=0.05), 
                                           WideResNetBlock(n),
                                           WideResNetBlock(n),
                                           WideResNetBlock(n),
                                           WideResNetBlock(n),                                                                                                                            
                                           stax.Relu(),
                                           stax.Dense(1, W_std=1.5, b_std=0.05))

apply_fn = jit(apply_fn)
kernel_fn = jit(kernel_fn, static_argnums=(2,))

In [17]:
predict_fn = nt.predict.gradient_descent_mse_ensemble(kernel_fn, X_train, 
                                                      y_train, diag_reg=1e-1)

ntk_prediction, _ = predict_fn(x_test=X_test, get='ntk', compute_cov=True)


from sklearn.metrics import mean_squared_error

print(mean_squared_error(ntk_prediction, y_test)/y_test.shape[0])

/usr/local/lib/python3.6/dist-packages/jax/_src/lax/lax.py:6081: UserWarning: Explicitly requested dtype <class 'jax._src.numpy.lax_numpy.float64'> requested in astype is not available, and will be truncated to dtype float32. To enable more dtypes, set the jax_enable_x64 configuration option or the JAX_ENABLE_X64 shell environment variable. See https://github.com/google/jax#current-gotchas for more.
  warnings.warn(msg.format(dtype, fun_name , truncated_dtype))
/usr/local/lib/python3.6/dist-packages/neural_tangents/stax.py:3926: UserWarning: `FanIn` layers assume independent inputs which is not verified in the code. Please make sure to have at least one `Dense` / `Conv` / `GlobalSelfAttention` etc. layer in each branch.
  warnings.warn('`FanIn` layers assume independent inputs which is not verified'


0.7307084853207653


In [18]:
from neural_tangents import stax

n = 2048


Main1 = stax.serial(stax.Relu(), stax.Dense(n, W_std=1.5, b_std=0.05))

Main2 = stax.serial(Main1, stax.Relu(), stax.Dense(n, W_std=1.5, b_std=0.05))

Main3 = stax.serial(Main2, stax.Relu(), stax.Dense(n, W_std=1.5, b_std=0.05))

Main4 = stax.serial(Main3, stax.Relu(), stax.Dense(n, W_std=1.5, b_std=0.05))

S3Net =  stax.serial(stax.FanOut(4), stax.parallel(Main1, Main2, Main3, Main4), stax.FanInSum())

init_fn, apply_fn, kernel_fn =  stax.serial(stax.Dense(n, W_std=1.5, b_std=0.05), S3Net,
                                            stax.Relu(),
                                            stax.Dense(1, W_std=1.5, b_std=0.05))

apply_fn = jit(apply_fn)
kernel_fn = jit(kernel_fn, static_argnums=(2,))

In [19]:
predict_fn = nt.predict.gradient_descent_mse_ensemble(kernel_fn, X_train, 
                                                      y_train, diag_reg=1e-1)

ntk_prediction, _ = predict_fn(x_test=X_test, get='ntk', compute_cov=True)


from sklearn.metrics import mean_squared_error

print(mean_squared_error(ntk_prediction, y_test)/y_test.shape[0])

/usr/local/lib/python3.6/dist-packages/jax/_src/lax/lax.py:6081: UserWarning: Explicitly requested dtype <class 'jax._src.numpy.lax_numpy.float64'> requested in astype is not available, and will be truncated to dtype float32. To enable more dtypes, set the jax_enable_x64 configuration option or the JAX_ENABLE_X64 shell environment variable. See https://github.com/google/jax#current-gotchas for more.
  warnings.warn(msg.format(dtype, fun_name , truncated_dtype))
/usr/local/lib/python3.6/dist-packages/neural_tangents/stax.py:3926: UserWarning: `FanIn` layers assume independent inputs which is not verified in the code. Please make sure to have at least one `Dense` / `Conv` / `GlobalSelfAttention` etc. layer in each branch.
  warnings.warn('`FanIn` layers assume independent inputs which is not verified'


0.7034404905251506


In [26]:
from neural_tangents import stax

n = 2048

def WideResNetBlock(n):
   Main = stax.serial(stax.Relu(), stax.Dense(n, W_std=1.5, b_std=0.05))
   Shortcut = stax.Identity()
   return stax.serial(stax.FanOut(2),
                     stax.parallel(Main, Shortcut),
                     stax.FanInSum())

init_fn, apply_fn, kernel_fn = stax.serial(stax.Dense(n, W_std=1.5, b_std=0.05), 
                                           WideResNetBlock(n),
                                           WideResNetBlock(n),
                                           WideResNetBlock(n),
                                           WideResNetBlock(n),  
                                           WideResNetBlock(n),                                                                                                                                                                       
                                           stax.Relu(),
                                           stax.Dense(1, W_std=1.5, b_std=0.05))

apply_fn = jit(apply_fn)
kernel_fn = jit(kernel_fn, static_argnums=(2,))


In [27]:
predict_fn = nt.predict.gradient_descent_mse_ensemble(kernel_fn, X_train, 
                                                      y_train, diag_reg=1e-1)

ntk_prediction, _ = predict_fn(x_test=X_test, get='ntk', compute_cov=True)


from sklearn.metrics import mean_squared_error

print(mean_squared_error(ntk_prediction, y_test)/y_test.shape[0])

/usr/local/lib/python3.6/dist-packages/jax/_src/lax/lax.py:6081: UserWarning: Explicitly requested dtype <class 'jax._src.numpy.lax_numpy.float64'> requested in astype is not available, and will be truncated to dtype float32. To enable more dtypes, set the jax_enable_x64 configuration option or the JAX_ENABLE_X64 shell environment variable. See https://github.com/google/jax#current-gotchas for more.
  warnings.warn(msg.format(dtype, fun_name , truncated_dtype))
/usr/local/lib/python3.6/dist-packages/neural_tangents/stax.py:3926: UserWarning: `FanIn` layers assume independent inputs which is not verified in the code. Please make sure to have at least one `Dense` / `Conv` / `GlobalSelfAttention` etc. layer in each branch.
  warnings.warn('`FanIn` layers assume independent inputs which is not verified'


0.7195870448042047


In [32]:
from neural_tangents import stax

n = 2048


Main1 = stax.serial(stax.Relu(), stax.Dense(n, W_std=1.5, b_std=0.05))

Main2 = stax.serial(Main1, stax.Relu(), stax.Dense(n, W_std=1.5, b_std=0.05))

Main3 = stax.serial(Main2, stax.Relu(), stax.Dense(n, W_std=1.5, b_std=0.05))

Main4 = stax.serial(Main3, stax.Relu(), stax.Dense(n, W_std=1.5, b_std=0.05))

Main5 = stax.serial(Main4, stax.Relu(), stax.Dense(n, W_std=1.5, b_std=0.05))

S3Net =  stax.serial(stax.FanOut(5), stax.parallel(Main1, Main2, Main3, Main4, Main5), stax.FanInSum())

init_fn, apply_fn, kernel_fn =  stax.serial(stax.Dense(n, W_std=1.5, b_std=0.05), S3Net,
                                            stax.Relu(),
                                            stax.Dense(1, W_std=1.5, b_std=0.05))

apply_fn = jit(apply_fn)
kernel_fn = jit(kernel_fn, static_argnums=(2,))

In [33]:
predict_fn = nt.predict.gradient_descent_mse_ensemble(kernel_fn, X_train, 
                                                      y_train, diag_reg=1e-1)

ntk_prediction, _ = predict_fn(x_test=X_test, get='ntk', compute_cov=True)


from sklearn.metrics import mean_squared_error

print(mean_squared_error(ntk_prediction, y_test)/y_test.shape[0])

/usr/local/lib/python3.6/dist-packages/jax/_src/lax/lax.py:6081: UserWarning: Explicitly requested dtype <class 'jax._src.numpy.lax_numpy.float64'> requested in astype is not available, and will be truncated to dtype float32. To enable more dtypes, set the jax_enable_x64 configuration option or the JAX_ENABLE_X64 shell environment variable. See https://github.com/google/jax#current-gotchas for more.
  warnings.warn(msg.format(dtype, fun_name , truncated_dtype))
/usr/local/lib/python3.6/dist-packages/neural_tangents/stax.py:3926: UserWarning: `FanIn` layers assume independent inputs which is not verified in the code. Please make sure to have at least one `Dense` / `Conv` / `GlobalSelfAttention` etc. layer in each branch.
  warnings.warn('`FanIn` layers assume independent inputs which is not verified'


0.684629923099064


In [20]:
from neural_tangents import stax

n = 2048

def WideResNetBlock(n):
   Main = stax.serial(stax.Relu(), stax.Dense(n, W_std=1.5, b_std=0.05))
   Shortcut = stax.Identity()
   return stax.serial(stax.FanOut(2),
                     stax.parallel(Main, Shortcut),
                     stax.FanInSum())

init_fn, apply_fn, kernel_fn = stax.serial(stax.Dense(n, W_std=1.5, b_std=0.05), 
                                           WideResNetBlock(n),
                                           WideResNetBlock(n),
                                           WideResNetBlock(n),
                                           WideResNetBlock(n),  
                                           WideResNetBlock(n),  
                                           WideResNetBlock(n),                                                                                                                                                                     
                                           stax.Relu(),
                                           stax.Dense(1, W_std=1.5, b_std=0.05))

apply_fn = jit(apply_fn)
kernel_fn = jit(kernel_fn, static_argnums=(2,))

In [21]:
predict_fn = nt.predict.gradient_descent_mse_ensemble(kernel_fn, X_train, 
                                                      y_train, diag_reg=1e-1)

ntk_prediction, _ = predict_fn(x_test=X_test, get='ntk', compute_cov=True)


from sklearn.metrics import mean_squared_error

print(mean_squared_error(ntk_prediction, y_test)/y_test.shape[0])

/usr/local/lib/python3.6/dist-packages/jax/_src/lax/lax.py:6081: UserWarning: Explicitly requested dtype <class 'jax._src.numpy.lax_numpy.float64'> requested in astype is not available, and will be truncated to dtype float32. To enable more dtypes, set the jax_enable_x64 configuration option or the JAX_ENABLE_X64 shell environment variable. See https://github.com/google/jax#current-gotchas for more.
  warnings.warn(msg.format(dtype, fun_name , truncated_dtype))
/usr/local/lib/python3.6/dist-packages/neural_tangents/stax.py:3926: UserWarning: `FanIn` layers assume independent inputs which is not verified in the code. Please make sure to have at least one `Dense` / `Conv` / `GlobalSelfAttention` etc. layer in each branch.
  warnings.warn('`FanIn` layers assume independent inputs which is not verified'


0.7073712491022515


In [37]:
from neural_tangents import stax

n = 2048


Main1 = stax.serial(stax.Relu(), stax.Dense(n, W_std=1.5, b_std=0.05))

Main2 = stax.serial(Main1, stax.Relu(), stax.Dense(n, W_std=1.5, b_std=0.05))

Main3 = stax.serial(Main2, stax.Relu(), stax.Dense(n, W_std=1.5, b_std=0.05))

Main4 = stax.serial(Main3, stax.Relu(), stax.Dense(n, W_std=1.5, b_std=0.05))

Main5 = stax.serial(Main4, stax.Relu(), stax.Dense(n, W_std=1.5, b_std=0.05))

Main6 = stax.serial(Main5, stax.Relu(), stax.Dense(n, W_std=1.5, b_std=0.05))

S3Net =  stax.serial(stax.FanOut(6), stax.parallel(Main1, Main2, Main3, Main4, Main5, Main6), stax.FanInSum())

init_fn, apply_fn, kernel_fn =  stax.serial(stax.Dense(n, W_std=1.5, b_std=0.05), S3Net,
                                            stax.Relu(),
                                            stax.Dense(1, W_std=1.5, b_std=0.05))

apply_fn = jit(apply_fn)
kernel_fn = jit(kernel_fn, static_argnums=(2,))

In [38]:
predict_fn = nt.predict.gradient_descent_mse_ensemble(kernel_fn, X_train, 
                                                      y_train, diag_reg=1e-1)

ntk_prediction, _ = predict_fn(x_test=X_test, get='ntk', compute_cov=True)


from sklearn.metrics import mean_squared_error

print(mean_squared_error(ntk_prediction, y_test)/y_test.shape[0])

/usr/local/lib/python3.6/dist-packages/jax/_src/lax/lax.py:6081: UserWarning: Explicitly requested dtype <class 'jax._src.numpy.lax_numpy.float64'> requested in astype is not available, and will be truncated to dtype float32. To enable more dtypes, set the jax_enable_x64 configuration option or the JAX_ENABLE_X64 shell environment variable. See https://github.com/google/jax#current-gotchas for more.
  warnings.warn(msg.format(dtype, fun_name , truncated_dtype))
/usr/local/lib/python3.6/dist-packages/neural_tangents/stax.py:3926: UserWarning: `FanIn` layers assume independent inputs which is not verified in the code. Please make sure to have at least one `Dense` / `Conv` / `GlobalSelfAttention` etc. layer in each branch.
  warnings.warn('`FanIn` layers assume independent inputs which is not verified'


0.6687406757873919


In [39]:
from neural_tangents import stax

n = 2048

def WideResNetBlock(n):
   Main = stax.serial(stax.Relu(), stax.Dense(n, W_std=1.5, b_std=0.05))
   Shortcut = stax.Identity()
   return stax.serial(stax.FanOut(2),
                     stax.parallel(Main, Shortcut),
                     stax.FanInSum())

init_fn, apply_fn, kernel_fn = stax.serial(stax.Dense(n, W_std=1.5, b_std=0.05), 
                                           WideResNetBlock(n),
                                           WideResNetBlock(n),
                                           WideResNetBlock(n),
                                           WideResNetBlock(n),  
                                           WideResNetBlock(n),  
                                           WideResNetBlock(n),   
                                           WideResNetBlock(n),                                                                                                                                                                                                               
                                           stax.Relu(),
                                           stax.Dense(1, W_std=1.5, b_std=0.05))

apply_fn = jit(apply_fn)
kernel_fn = jit(kernel_fn, static_argnums=(2,))

In [40]:
predict_fn = nt.predict.gradient_descent_mse_ensemble(kernel_fn, X_train, 
                                                      y_train, diag_reg=1e-1)

ntk_prediction, _ = predict_fn(x_test=X_test, get='ntk', compute_cov=True)


from sklearn.metrics import mean_squared_error

print(mean_squared_error(ntk_prediction, y_test)/y_test.shape[0])

/usr/local/lib/python3.6/dist-packages/jax/_src/lax/lax.py:6081: UserWarning: Explicitly requested dtype <class 'jax._src.numpy.lax_numpy.float64'> requested in astype is not available, and will be truncated to dtype float32. To enable more dtypes, set the jax_enable_x64 configuration option or the JAX_ENABLE_X64 shell environment variable. See https://github.com/google/jax#current-gotchas for more.
  warnings.warn(msg.format(dtype, fun_name , truncated_dtype))
/usr/local/lib/python3.6/dist-packages/neural_tangents/stax.py:3926: UserWarning: `FanIn` layers assume independent inputs which is not verified in the code. Please make sure to have at least one `Dense` / `Conv` / `GlobalSelfAttention` etc. layer in each branch.
  warnings.warn('`FanIn` layers assume independent inputs which is not verified'


0.6975477101686665


In [41]:
from neural_tangents import stax

n = 2048


Main1 = stax.serial(stax.Relu(), stax.Dense(n, W_std=1.5, b_std=0.05))

Main2 = stax.serial(Main1, stax.Relu(), stax.Dense(n, W_std=1.5, b_std=0.05))

Main3 = stax.serial(Main2, stax.Relu(), stax.Dense(n, W_std=1.5, b_std=0.05))

Main4 = stax.serial(Main3, stax.Relu(), stax.Dense(n, W_std=1.5, b_std=0.05))

Main5 = stax.serial(Main4, stax.Relu(), stax.Dense(n, W_std=1.5, b_std=0.05))

Main6 = stax.serial(Main5, stax.Relu(), stax.Dense(n, W_std=1.5, b_std=0.05))

Main7 = stax.serial(Main6, stax.Relu(), stax.Dense(n, W_std=1.5, b_std=0.05))

S3Net =  stax.serial(stax.FanOut(7), stax.parallel(Main1, Main2, Main3, Main4, Main5, Main6, Main7), stax.FanInSum())

init_fn, apply_fn, kernel_fn =  stax.serial(stax.Dense(n, W_std=1.5, b_std=0.05), S3Net,
                                            stax.Relu(),
                                            stax.Dense(1, W_std=1.5, b_std=0.05))

apply_fn = jit(apply_fn)
kernel_fn = jit(kernel_fn, static_argnums=(2,))

In [42]:
predict_fn = nt.predict.gradient_descent_mse_ensemble(kernel_fn, X_train, 
                                                      y_train, diag_reg=1e-1)

ntk_prediction, _ = predict_fn(x_test=X_test, get='ntk', compute_cov=True)


from sklearn.metrics import mean_squared_error

print(mean_squared_error(ntk_prediction, y_test)/y_test.shape[0])

/usr/local/lib/python3.6/dist-packages/jax/_src/lax/lax.py:6081: UserWarning: Explicitly requested dtype <class 'jax._src.numpy.lax_numpy.float64'> requested in astype is not available, and will be truncated to dtype float32. To enable more dtypes, set the jax_enable_x64 configuration option or the JAX_ENABLE_X64 shell environment variable. See https://github.com/google/jax#current-gotchas for more.
  warnings.warn(msg.format(dtype, fun_name , truncated_dtype))
/usr/local/lib/python3.6/dist-packages/neural_tangents/stax.py:3926: UserWarning: `FanIn` layers assume independent inputs which is not verified in the code. Please make sure to have at least one `Dense` / `Conv` / `GlobalSelfAttention` etc. layer in each branch.
  warnings.warn('`FanIn` layers assume independent inputs which is not verified'


0.656763956903401


In [43]:
from neural_tangents import stax

n = 2048

def WideResNetBlock(n):
   Main = stax.serial(stax.Relu(), stax.Dense(n, W_std=1.5, b_std=0.05))
   Shortcut = stax.Identity()
   return stax.serial(stax.FanOut(2),
                     stax.parallel(Main, Shortcut),
                     stax.FanInSum())

init_fn, apply_fn, kernel_fn = stax.serial(stax.Dense(n, W_std=1.5, b_std=0.05), 
                                           WideResNetBlock(n),
                                           WideResNetBlock(n),
                                           WideResNetBlock(n),
                                           WideResNetBlock(n),  
                                           WideResNetBlock(n),  
                                           WideResNetBlock(n),   
                                           WideResNetBlock(n),    
                                           WideResNetBlock(n),                                                                                                                                                                                                             
                                           stax.Relu(),
                                           stax.Dense(1, W_std=1.5, b_std=0.05))

apply_fn = jit(apply_fn)
kernel_fn = jit(kernel_fn, static_argnums=(2,))

In [44]:
predict_fn = nt.predict.gradient_descent_mse_ensemble(kernel_fn, X_train, 
                                                      y_train, diag_reg=1e-1)

ntk_prediction, _ = predict_fn(x_test=X_test, get='ntk', compute_cov=True)


from sklearn.metrics import mean_squared_error

print(mean_squared_error(ntk_prediction, y_test)/y_test.shape[0])

/usr/local/lib/python3.6/dist-packages/jax/_src/lax/lax.py:6081: UserWarning: Explicitly requested dtype <class 'jax._src.numpy.lax_numpy.float64'> requested in astype is not available, and will be truncated to dtype float32. To enable more dtypes, set the jax_enable_x64 configuration option or the JAX_ENABLE_X64 shell environment variable. See https://github.com/google/jax#current-gotchas for more.
  warnings.warn(msg.format(dtype, fun_name , truncated_dtype))
/usr/local/lib/python3.6/dist-packages/neural_tangents/stax.py:3926: UserWarning: `FanIn` layers assume independent inputs which is not verified in the code. Please make sure to have at least one `Dense` / `Conv` / `GlobalSelfAttention` etc. layer in each branch.
  warnings.warn('`FanIn` layers assume independent inputs which is not verified'


0.6902528734078427


In [45]:
from neural_tangents import stax

n = 2048


Main1 = stax.serial(stax.Relu(), stax.Dense(n, W_std=1.5, b_std=0.05))

Main2 = stax.serial(Main1, stax.Relu(), stax.Dense(n, W_std=1.5, b_std=0.05))

Main3 = stax.serial(Main2, stax.Relu(), stax.Dense(n, W_std=1.5, b_std=0.05))

Main4 = stax.serial(Main3, stax.Relu(), stax.Dense(n, W_std=1.5, b_std=0.05))

Main5 = stax.serial(Main4, stax.Relu(), stax.Dense(n, W_std=1.5, b_std=0.05))

Main6 = stax.serial(Main5, stax.Relu(), stax.Dense(n, W_std=1.5, b_std=0.05))

Main7 = stax.serial(Main6, stax.Relu(), stax.Dense(n, W_std=1.5, b_std=0.05))

Main8 = stax.serial(Main7, stax.Relu(), stax.Dense(n, W_std=1.5, b_std=0.05))

S3Net =  stax.serial(stax.FanOut(8), stax.parallel(Main1, Main2, Main3, Main4, Main5, Main6, Main7, Main8), stax.FanInSum())

init_fn, apply_fn, kernel_fn =  stax.serial(stax.Dense(n, W_std=1.5, b_std=0.05), S3Net,
                                            stax.Relu(),
                                            stax.Dense(1, W_std=1.5, b_std=0.05))

apply_fn = jit(apply_fn)
kernel_fn = jit(kernel_fn, static_argnums=(2,))

In [46]:
predict_fn = nt.predict.gradient_descent_mse_ensemble(kernel_fn, X_train, 
                                                      y_train, diag_reg=1e-1)

ntk_prediction, _ = predict_fn(x_test=X_test, get='ntk', compute_cov=True)


from sklearn.metrics import mean_squared_error

print(mean_squared_error(ntk_prediction, y_test)/y_test.shape[0])

/usr/local/lib/python3.6/dist-packages/jax/_src/lax/lax.py:6081: UserWarning: Explicitly requested dtype <class 'jax._src.numpy.lax_numpy.float64'> requested in astype is not available, and will be truncated to dtype float32. To enable more dtypes, set the jax_enable_x64 configuration option or the JAX_ENABLE_X64 shell environment variable. See https://github.com/google/jax#current-gotchas for more.
  warnings.warn(msg.format(dtype, fun_name , truncated_dtype))
/usr/local/lib/python3.6/dist-packages/neural_tangents/stax.py:3926: UserWarning: `FanIn` layers assume independent inputs which is not verified in the code. Please make sure to have at least one `Dense` / `Conv` / `GlobalSelfAttention` etc. layer in each branch.
  warnings.warn('`FanIn` layers assume independent inputs which is not verified'


0.6512351108924324
